# Лабораторная работа №5

### 1. Хеш-функции

**Хеш-функция** — это функция, которая преобразует входные данные произвольного размера (например, строки или числа) в фиксированный размер (обычно число). Хеш-функции часто используются для хранения данных в хеш-таблицах и должны обладать следующими свойствами:

- **Детерминированность**: для одного и того же входа всегда возвращается одно и то же значение.
- **Быстрое выполнение**: функция должна работать за константное время.
- **Минимизация коллизий**: желательно, чтобы разные входные данные имели разные хеш-значения.
- **Равномерное распределение**: результат работы функции должен быть равномерно распределен по диапазону возможных значений.

Примеры хеш-функций:
- CRC32
- MD5
- SHA-1 и SHA-256

### 2. Хеш-таблицы

**Хеш-таблица** — это структура данных, которая использует хеш-функцию для маппинга ключей на индексы в массиве. Основное назначение хеш-таблицы — это хранение данных с быстрым доступом, добавлением и удалением за амортизированное константное время \(O(1)\), если хеш-функция хорошо справляется с распределением ключей.

Хеш-таблицы состоят из:
- **Массивов** для хранения значений.
- **Хеш-функций** для вычисления индексов, по которым размещаются данные.

#### Основные операции над хеш-таблицами:
1. **Вставка (Insert)**: добавление нового ключа-значения. Вычисляется хеш от ключа, и элемент помещается в соответствующую ячейку массива. При коллизии (если два разных ключа попадают в один и тот же индекс) используются методы разрешения коллизий.
  
2. **Поиск (Search)**: нахождение значения по ключу. Вычисляется хеш-значение для ключа, после чего производится проверка ячейки на наличие нужного ключа.
  
3. **Удаление (Delete)**: удаление пары ключ-значение по ключу. Вычисляется индекс, проверяется наличие ключа, после чего элемент удаляется из таблицы.

### 3. Методы разрешения коллизий

Коллизии возникают, когда хеш-функция возвращает одинаковое значение для двух различных ключей. Основные методы разрешения коллизий:
- **Открытая адресация**: при коллизии алгоритм ищет следующий свободный слот. Наиболее распространенные методы — линейное, квадратичное и двойное хеширование.
- **Цепочки (Chaining)**: каждый индекс массива содержит связный список. Если для одного индекса имеется несколько ключей, то они добавляются в связный список.

### Преимущества и недостатки хеш-таблиц

**Преимущества**:
- Очень высокая скорость доступа к данным.
- Простота реализации и эффективное использование памяти.

**Недостатки**:
- Потенциальные проблемы с коллизиями, которые могут снизить производительность.
- Зависимость от качества хеш-функции и методов разрешения коллизий.

Изучение хеш-функций и хеш-таблиц предоставляет мощные инструменты для оптимизации программ, так как позволяет хранить и извлекать данные с минимальными затратами по времени.

---

# Задача 1: Реализовать хеш-таблицу на основе цепочек (Chaining)

In [1]:
class ChainingHashTable:
    def __init__(self, size):
        self.size = size
        self.table = [[] for _ in range(size)]

    def hash_function(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        index = self.hash_function(key)
        for pair in self.table[index]:
            if pair[0] == key:
                pair[1] = value  # Обновление значения
                return
        self.table[index].append([key, value])

    def get(self, key):
        index = self.hash_function(key)
        for pair in self.table[index]:
            if pair[0] == key:
                return pair[1]
        return None

    def delete(self, key):
        index = self.hash_function(key)
        for i, pair in enumerate(self.table[index]):
            if pair[0] == key:
                del self.table[index][i]
                return True
        return False


# Задача 2: Реализовать хеш-таблицу на основе открытой адресации

In [ ]:
class OpenAddressingHashTable:
    def __init__(self, size):
        self.size = size
        self.table = [None] * size

    def hash_function(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        index = self.hash_function(key)
        start_index = index
        while self.table[index] is not None:
            if self.table[index][0] == key:
                self.table[index] = (key, value)  # Обновление значения
                return
            index = (index + 1) % self.size
            if index == start_index:
                raise Exception("Hash table is full")
        self.table[index] = (key, value)

    def get(self, key):
        index = self.hash_function(key)
        start_index = index
        while self.table[index] is not None:
            if self.table[index][0] == key:
                return self.table[index][1]
            index = (index + 1) % self.size
            if index == start_index:
                break
        return None

    def delete(self, key):
        index = self.hash_function(key)
        start_index = index
        while self.table[index] is not None:
            if self.table[index][0] == key:
                self.table[index] = None
                return True
            index = (index + 1) % self.size
            if index == start_index:
                break
        return False


# Задача 3: Блокчейн

In [ ]:
import hashlib

class Block:
    def __init__(self, index, previous_hash, data):
        self.index = index
        self.previous_hash = previous_hash
        self.data = data
        self.hash = self.compute_hash()

    def compute_hash(self):
        block_string = f"{self.index}{self.previous_hash}{self.data}"
        return hashlib.sha256(block_string.encode()).hexdigest()

class Blockchain:
    def __init__(self):
        self.chain = []
        self.create_block(data="Genesis Block", previous_hash="0")

    def create_block(self, data, previous_hash):
        index = len(self.chain)
        new_block = Block(index, previous_hash, data)
        self.chain.append(new_block)
        return new_block

    def get_last_block(self):
        return self.chain[-1] if self.chain else None

    def add_block(self, data):
        last_block = self.get_last_block()
        previous_hash = last_block.hash if last_block else "0"
        new_block = self.create_block(data, previous_hash)
        return new_block


# Задача 4: Проверка пересечения двух массивов

In [ ]:
def has_intersection(arr1, arr2):
    elements = set(arr1)
    for num in arr2:
        if num in elements:
            return True
    return False


# Задача 5: Проверка уникальности элементов в массиве

In [ ]:
def all_unique(arr):
    seen = set()
    for num in arr:
        if num in seen:
            return False
        seen.add(num)
    return True


# Задача 6: Нахождение пар с заданной суммой

In [ ]:
def find_pairs_with_sum(arr, target_sum):
    result = []
    seen = set()
    for num in arr:
        complement = target_sum - num
        if complement in seen:
            result.append((complement, num))
        seen.add(num)
    return result


# Задача 7: Проверка анаграмм

In [ ]:
def are_anagrams(str1, str2):
    if len(str1) != len(str2):
        return False
    count1, count2 = {}, {}
    for char in str1:
        count1[char] = count1.get(char, 0) + 1
    for char in str2:
        count2[char] = count2.get(char, 0) + 1
    return count1 == count2


# Ответы на контрольные вопросы:

---

### 1. Назначение хеширования

Хеширование — это процесс преобразования данных (например, строки, числа или других объектов) в фиксированный размер хеш-значения, которое служит уникальным "идентификатором" этих данных. Основные цели и задачи хеширования:
- **Эффективный доступ к данным**: Позволяет организовать быстрый доступ к данным в хеш-таблице, где поиск, добавление и удаление элементов может выполняться за время \(O(1)\) в среднем.
- **Сравнение данных**: Ускоряет операции сравнения больших объемов данных, так как их можно сравнивать по хеш-значениям.
- **Целостность данных**: Хеширование помогает проверять целостность данных, что широко используется, например, в криптографии и блокчейне для проверки подлинности и защиты от подделок.

### 2. Способы реализации хеш-функций и хеш-таблиц

**Хеш-функция** — это функция, которая преобразует входные данные в хеш-значение (целое число, обычно фиксированной длины). Способы реализации хеш-функций:
- **Деление по модулю**: Хеш-значение вычисляется как остаток от деления на размер таблицы. Пример: `h(x) = x % N`, где `N` — размер хеш-таблицы.
- **Метод умножения**: Входное значение умножается на константу, и дробная часть результата используется для определения индекса.
- **Сложение и битовые сдвиги**: Входные данные обрабатываются побитово с использованием операций сдвига и XOR, что дает равномерное распределение.
- **Криптографические хеш-функции**: Такие функции, как SHA-256, создают уникальные хеши для защищенных операций. Они устойчивы к коллизиям, но работают медленнее.

**Хеш-таблицы** — структуры данных для хранения пар "ключ-значение". Основные методы реализации:
- **Метод цепочек**: Для каждой позиции таблицы создается список (цепочка), где хранятся все элементы с одинаковым хеш-значением.
- **Открытая адресация**: Если место занято, используется альтернативный индекс для поиска свободного места (линейное пробирование, квадратичное пробирование и двойное хеширование).

### 3. Понятие коллизии

**Коллизия** — это ситуация, когда хеш-функция сопоставляет разные ключи с одним и тем же индексом (хеш-значением) в хеш-таблице. Коллизии неизбежны, так как количество возможных ключей обычно намного больше, чем размер хеш-таблицы. Правильное разрешение коллизий — важная задача при проектировании хеш-таблиц.

### 4. Варианты разрешения коллизий в хеш-таблице

Существуют несколько способов разрешения коллизий:

- **Метод цепочек (chaining)**: Каждая ячейка хеш-таблицы содержит указатель на список или связный список всех элементов, которые имеют один и тот же индекс. Если происходит коллизия, элемент добавляется в список по соответствующему индексу. Преимущество метода — возможность хранения неограниченного числа элементов, но при большом количестве коллизий скорость доступа к элементам снижается.

- **Открытая адресация (open addressing)**: Все элементы хранятся непосредственно в массиве хеш-таблицы, и при коллизии производится поиск свободного места по определенному алгоритму. Существуют несколько методов:
  - **Линейное пробирование**: если место занято, следующий элемент помещается на первую свободную ячейку, идущую подряд.
  - **Квадратичное пробирование**: используется квадратное смещение (1, 4, 9, ...) для поиска свободной ячейки.
  - **Двойное хеширование**: при коллизии вычисляется новое хеш-значение по другой хеш-функции.

- **Кукушечное хеширование**: использует две или более хеш-функции. Если при добавлении возникает коллизия, элемент "выбивает" существующий элемент, и последний перемещается по другой хеш-функции.

- **Рехеширование (rehashing)**: при достижении определенной загрузки таблицы (например, 70%) создается новая, более крупная хеш-таблица, и элементы копируются в нее с новыми индексами.